## ** To access the layers in the map click the grey layers button on the left.This is a double choropleth map with visualization for two metrics  ** ##



## Visual Intent :

Folium maps have excellent visual appeal. One can zoom into details of the maps seamlessly as well layer multiple maps in a single map.The positional markers tell counts of properties in that area and zooming in shows various levels of details in the map.

The below map is a double choropleth map with  visulaization for two metrics :

1) Payback Period
2) Roi with equity


## Business Intent: 

To get an intuitive idea on high/low returns regions for 2-bedroom properties in NY based in the  2 metrics in the study.


## Key Insights

Zipcodes in King's county will have high ROI in next 10 years 
Zipcodes in NY county have high payback periods

## Metrics : ( For detailed explanation please check documentation) 

Payback Period: A conservative estimate of how much time it would take to pay off rent assuming annual rent remains constant.High annual rents would have low payback value and vice versa.

ROI_10 : An optimitic estimate that  factors the value of equity and rent in calculating roi and also factors an optimistic appreciation rate for both rent and equity.





In [1]:
import pandas as pd
import folium
data1=pd.read_pickle("pickles/air_zillgroup.pkl")
import urllib.request, json 
with urllib.request.urlopen("https://raw.githubusercontent.com/fedhere/PUI2015_EC/master/mam1612_EC/nyc-zip-code-tabulation-areas-polygons.geojson") as url:
    data_geo = json.loads(url.read().decode())


In [2]:
data1=data1.groupby(["zipcode"],as_index=False).mean()[["longitude","latitude","zipcode","payback_period","sell_price","annual_rent","roi_equity","roi"]]
county=pd.read_csv('https://query.data.world/s/buotxun7n3q6twzd6havwil7h5nqpb')
county["ZIP"]=county["ZIP"].astype(str)
data1=data1.merge(county,how="left",left_on="zipcode",right_on="ZIP")
tmp=data_geo
geozips = []
for i in range(len(tmp['features'])):
    if tmp['features'][i]['properties']['postalCode'] in list(data1['zipcode'].unique()):
        geozips.append(tmp['features'][i])

new_json = dict.fromkeys(['type','features'])
new_json['type'] = 'FeatureCollection'
new_json['features'] = geozips
king_geo=open("cleaned_geodata.json", "w").write(json.dumps(new_json, sort_keys=True, indent=4, separators=(',', ': ')))


In [3]:
from folium import FeatureGroup, LayerControl, Map, Marker
king_geo= "cleaned_geodata.json"
from folium.plugins import MarkerCluster

map = folium.Map(location=[40.693943, -73.985880], default_zoom_start=15,tiles="cartodbpositron")

k=folium.Choropleth(geo_data=king_geo,
             data=data1, # my dataset
             columns=['zipcode', 'payback_period'], # zip code is here for matching the geojson zipcode, sales price is the column that changes the color of zipcode areas
             key_on='feature.properties.postalCode', # this path contains zipcodes in str type, this zipcodes should match with our ZIP CODE column
             fill_color='YlOrRd', fill_opacity=0.8, line_opacity=0.8,
             legend_name='payback_period',name="payback_period").add_to(map)

marker_cluster = MarkerCluster().add_to(k)
for i in range(data1.shape[0]):
    location = [data1['latitude'][i],data1['longitude'][i]]
    tooltip = "Zipcode:{}<br> COUNTYNAME: {}<br> Click for more".format(data1["zipcode"][i],data1["COUNTYNAME"][i])
    
    folium.Marker(location, # adding more details to the popup screen using HTML
                  popup="""
                  <i>PAYBACK PERIOD: </i> <br> <b>{}</b> <br> 
                  <i>Annual Rent: </i><b><br>{}</b><br>
                  <i>Selling price: </i><b><br>{}</b><br>""".format(
                    round(data1['payback_period'][i],2), 
                    round(data1['annual_rent'][i],2), 
                    round(data1['sell_price'][i],2)), 
                  tooltip=tooltip).add_to(marker_cluster)


    
n=folium.Choropleth(geo_data=king_geo,
             data=data1, # my dataset
             columns=['zipcode', 'roi_equity'], # zip code is here for matching the geojson zipcode, sales price is the column that changes the color of zipcode areas
             key_on='feature.properties.postalCode', # this path contains zipcodes in str type, this zipcodes should match with our ZIP CODE column
             fill_color='PuRd', fill_opacity=0.8, line_opacity=0.8,
             legend_name='roi_equity',name="roi_equity").add_to(map)
marker_cluster2 = MarkerCluster().add_to(n)
for i in range(data1.shape[0]):
    location = [data1['latitude'][i],data1['longitude'][i]]
    tooltip = "Zipcode:{}<br> County: {}<br> Click for more".format(data1["zipcode"][i],data1["COUNTYNAME"][i])
    
    folium.Marker(location, # adding more details to the popup screen using HTML
                  popup="""
                  <i>County:</i> <br> <b>{}</b> <br> 
                  <i>ROI_10: </i><b><br>{}</b><br>
                  <i>payback_period: </i><b><br>{}</b><br>""".format(
                    data1['COUNTYNAME'][i], 
                    round(data1['annual_rent'][i],2), 
                    round(data1['payback_period'][i],2)), 
                  tooltip=tooltip).add_to(marker_cluster2)  

    

LayerControl().add_to(map)
map

In [4]:
map.save("app/maps.html")